In [6]:
import os
import torch
import json
import librosa
import accelerate
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, AutoTokenizer, AutoModelForCausalLM
from dotenv import load_dotenv
import pathlib
import textwrap
from anthropic import Anthropic


# hide sensible Claude API key information by importing environment variables  from an env.file
load_dotenv(r"anthropic_access.env")
# store the token in a variable
api_key= os.environ.get("ANTHROPIC_KEY")

# hide sensible HuggingFace token information by importing environment variables from an env. file
load_dotenv(r"huggingface.env")
# store the token in a variable
token=os.environ.get("HF_TOKEN")

In [13]:
# load the whisper large model from HuggingFace

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True,)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# use a sample mp3 file, in this case a mp4 to mp3 converted youtube video explaining the key concepts of Langchain
sample= r"audio directory\The LangChain Cookbook - Beginner Guide To 7 Essential Concepts.mp3"
result= pipe(sample)
print(result["text"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 Hello good people! Have you ever wondered what Langchain was? Or maybe you've heard about it and you've played around with a few sections but you're not quite sure where to look next? Well in this video we're going to be covering all of the Langchain basics with the goal of getting you building and having fun as quick as possible. My name is Greg and I've been having a ton of fun building out apps in Langchain. Now, I share most of my work on Twitter, so if you wanna go check it out, links in the description, you can go follow along with me. Now, this video is gonna be based off of the new conceptual docs from Langchain. And the reason why I'm doing a video here is because it takes all of the technical pieces and abstracts them up into more theoretical, qualitative aspects of Langchain, which I think is extremely helpful for it. And in order to understand this a little bit better, I've created a companion for this video. And that is the Lane Chain Cookbook. Link's in the description. 

In [14]:
# summarize the transcripted result using the Anthropic API with Claude3 Opus, a state of the art LLM that is extremely fast and exerces excellence in all tasks
transcription= result["text"]

# use the Claude API to summarize the text with Claude 3 Sonnet due to its large output and input capablities
# Claude 3 Opus is also available but cost/million token is 5x higher than for Claude 3 Sonnet
# Pre-define the prompt for cleaner code

prompt="You are a fast learner getting started in the field of AI. Please summarize the key concepts of Langchain, following the transcript about a video explanation about the framework."\
"Highlight the key concepts and use headlines for every key concept you explain in the summary. Here is the transcript: {transcription}"

# Initialize the client
client= Anthropic(
    api_key=api_key,
)

#configure the model for message creation
message= client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=4096,
    messages=[
        {"role":"user","content":prompt}
    ]
)

# print only the text part of the response body which includes the summarization
print(message.content[0].text)

Here's a summary of the key concepts of Langchain based on the provided transcript, with headlines for each key concept:

#1 What is Langchain?
Langchain is a framework that helps developers build applications with large language models (LLMs) like GPT-3. It provides a modular and extensible approach to working with LLMs, making it easier to integrate them into various applications.

#2 Agents
Agents are a core concept in Langchain. They represent the ability to break down complex tasks into smaller steps, retrieve relevant information, and iteratively refine the solution. Agents can be thought of as autonomous entities that can reason and plan to achieve a specific goal.

#3 Tools
Tools are the building blocks that agents use to accomplish tasks. They can be anything from search engines, databases, APIs, or even other LLMs. Agents can leverage multiple tools to gather information, perform calculations, or execute actions.

#4 Memory
Memory is a crucial component that allows agents to 